# Phase 3: Migration Intelligence Scoring

## Objective
Convert engineered district-level features into an explainable migration
intensity score and category.

## Input
- `data/processed/district_features.csv` (district × month)

## Output
- `data/final/migration_intensity.csv`
- `data/final/migration_intensity.json`

## Migration Score Logic
The migration_score is a **relative** signal:
- For each month, compute the mean migration_index across ALL districts
- migration_score = district_migration_index - monthly_mean
- Positive = net inflow, Negative = net outflow

## Category Thresholds
| Score Range | Category |
|-------------|----------|
| > 0.015 | High Inflow |
| 0.00 to 0.015 | Moderate Inflow |
| -0.015 to 0.00 | Moderate Outflow |
| < -0.015 | High Outflow |

## Cell 1: Import Libraries and Set Paths

In [1]:
"""
Import Required Libraries
-------------------------
- pandas: Data manipulation
- json: JSON file output
- pathlib: Cross-platform file path handling
"""
import pandas as pd
import json
from pathlib import Path

# Define project paths
PROJECT_ROOT = Path.cwd().parent
INPUT_PATH = PROJECT_ROOT / "data" / "processed" / "district_features.csv"
OUTPUT_CSV_PATH = PROJECT_ROOT / "data" / "final" / "migration_intensity.csv"
OUTPUT_JSON_PATH = PROJECT_ROOT / "data" / "final" / "migration_intensity.json"

# Ensure output directory exists
OUTPUT_CSV_PATH.parent.mkdir(parents=True, exist_ok=True)

print(f"✅ Libraries imported")
print(f"📂 Input: {INPUT_PATH}")
print(f"📂 Output CSV: {OUTPUT_CSV_PATH}")
print(f"📂 Output JSON: {OUTPUT_JSON_PATH}")

✅ Libraries imported
📂 Input: d:\Projects\ML - DataScience\Saarthi-Net-Data-Pipeline\data\processed\district_features.csv
📂 Output CSV: d:\Projects\ML - DataScience\Saarthi-Net-Data-Pipeline\data\final\migration_intensity.csv
📂 Output JSON: d:\Projects\ML - DataScience\Saarthi-Net-Data-Pipeline\data\final\migration_intensity.json


## Cell 2: Load Feature Data from Phase 2

In [2]:
"""
Load Feature Data
-----------------
Read the district-month feature table from Phase 2.
We only need district_id, month, and migration_index for this phase.
"""
df = pd.read_csv(INPUT_PATH)

print(f"✅ Loaded {len(df):,} rows from Phase 2 output")
print(f"\n📊 Columns: {list(df.columns)}")
print(f"\n📋 Districts: {df['district_id'].nunique()}")
print(f"📋 Months: {df['month'].nunique()}")
print(f"\n📋 Migration index statistics:")
print(f"   Min: {df['migration_index'].min():.4f}")
print(f"   Max: {df['migration_index'].max():.4f}")
print(f"   Mean: {df['migration_index'].mean():.4f}")
print(f"\n📋 Sample rows:")
df[['district_id', 'month', 'migration_index']].head(10)

✅ Loaded 240 rows from Phase 2 output

📊 Columns: ['district_id', 'month', 'address_velocity', 'migration_index', 'youth_ratio', 'digital_gap', 'growth_3m_consistent']

📋 Districts: 20
📋 Months: 12

📋 Migration index statistics:
   Min: 0.0060
   Max: 0.0596
   Mean: 0.0307

📋 Sample rows:


,district_id,month,migration_index
0,GJ_AMD,2024-01,0.0282
1,GJ_AMD,2024-02,0.0451
2,GJ_AMD,2024-03,0.0368
3,GJ_AMD,2024-04,0.0327
4,GJ_AMD,2024-05,0.0593
5,GJ_AMD,2024-06,0.0519
6,GJ_AMD,2024-07,0.0349
7,GJ_AMD,2024-08,0.0457
8,GJ_AMD,2024-09,0.0534
9,GJ_AMD,2024-10,0.0272


## Cell 3: Compute Monthly Mean Migration Index

For each month, we compute the average migration_index across ALL districts.
This serves as the baseline for determining relative migration intensity.

In [3]:
"""
Compute Monthly Mean Migration Index
-------------------------------------
Step 1: Group all districts by month
Step 2: Calculate the mean migration_index for each month

This gives us a "national average" baseline for each month.
Districts above this mean are experiencing net inflow.
Districts below this mean are experiencing net outflow.
"""

# Calculate monthly mean migration_index across all districts
monthly_mean = df.groupby('month')['migration_index'].mean().reset_index()
monthly_mean.columns = ['month', 'monthly_mean_migration_index']

print(f"✅ Computed monthly mean migration index")
print(f"\n📊 Monthly baseline values:")
monthly_mean

✅ Computed monthly mean migration index

📊 Monthly baseline values:


,month,monthly_mean_migration_index
0,2024-01,0.026470
1,2024-02,0.029735
2,2024-03,0.026340
3,2024-04,0.032135
4,2024-05,0.034710
5,2024-06,0.031665
6,2024-07,0.029525
7,2024-08,0.031285
8,2024-09,0.032260
9,2024-10,0.029480


## Cell 4: Compute Migration Score

**Formula:** `migration_score = district_migration_index - monthly_mean`

**Interpretation:**
- **Positive score**: District has higher-than-average migration activity (net inflow)
- **Negative score**: District has lower-than-average migration activity (net outflow)
- **Zero**: District is at the monthly average

In [4]:
"""
Compute Migration Score (Relative Migration Signal)
----------------------------------------------------
The migration_score measures how much a district's migration activity
deviates from the monthly average across all districts.

Formula: migration_score = migration_index - monthly_mean_migration_index

This is a zero-centered score:
- Positive values indicate NET INFLOW (more migration than average)
- Negative values indicate NET OUTFLOW (less migration than average)
"""

# Merge monthly mean into main dataframe
df_scored = df.merge(monthly_mean, on='month', how='left')

# Calculate migration score as deviation from monthly mean
df_scored['migration_score'] = (
    df_scored['migration_index'] - df_scored['monthly_mean_migration_index']
)

# Round to 4 decimal places for cleaner output
df_scored['migration_score'] = df_scored['migration_score'].round(4)

print(f"✅ Migration scores computed")
print(f"\n📊 Score statistics:")
print(f"   Min: {df_scored['migration_score'].min():.4f}")
print(f"   Max: {df_scored['migration_score'].max():.4f}")
print(f"   Mean: {df_scored['migration_score'].mean():.4f} (should be ~0)")
print(f"\n📋 Sample scored rows:")
df_scored[['district_id', 'month', 'migration_index', 'monthly_mean_migration_index', 'migration_score']].head(12)

✅ Migration scores computed

📊 Score statistics:
   Min: -0.0271
   Max: 0.0259
   Mean: 0.0000 (should be ~0)

📋 Sample scored rows:


,district_id,month,migration_index,monthly_mean_migration_index,migration_score
0,GJ_AMD,2024-01,0.0282,0.026470,0.0017
1,GJ_AMD,2024-02,0.0451,0.029735,0.0154
2,GJ_AMD,2024-03,0.0368,0.026340,0.0105
3,GJ_AMD,2024-04,0.0327,0.032135,0.0006
4,GJ_AMD,2024-05,0.0593,0.034710,0.0246
5,GJ_AMD,2024-06,0.0519,0.031665,0.0202
6,GJ_AMD,2024-07,0.0349,0.029525,0.0054
7,GJ_AMD,2024-08,0.0457,0.031285,0.0144
8,GJ_AMD,2024-09,0.0534,0.032260,0.0211
9,GJ_AMD,2024-10,0.0272,0.029480,-0.0023


## Cell 5: Assign Migration Category

**Category Rules:**
| Score Range | Category |
|-------------|----------|
| > 0.015 | High Inflow |
| 0.00 to 0.015 | Moderate Inflow |
| -0.015 to 0.00 | Moderate Outflow |
| < -0.015 | High Outflow |

*Note: Using 0.015 thresholds based on actual data distribution (scores range ~±0.03)*

In [5]:
"""
Assign Migration Category
-------------------------
Categorize districts based on their migration_score.

Thresholds are set at ±0.015 based on the data distribution:
- Scores range from approximately -0.025 to +0.030
- 0.015 represents roughly half a standard deviation

Categories:
- High Inflow: Significantly above average (strong destination)
- Moderate Inflow: Slightly above average
- Moderate Outflow: Slightly below average
- High Outflow: Significantly below average (strong source)
"""

def assign_migration_category(score: float) -> str:
    """
    Assign a human-readable category based on migration score.
    
    Args:
        score: The migration_score (deviation from monthly mean)
    
    Returns:
        Category string: one of 4 categories
    """
    if score > 0.015:
        return "High Inflow"
    elif score >= 0.0:
        return "Moderate Inflow"
    elif score >= -0.015:
        return "Moderate Outflow"
    else:
        return "High Outflow"

# Apply category assignment
df_scored['migration_category'] = df_scored['migration_score'].apply(assign_migration_category)

print(f"✅ Migration categories assigned")
print(f"\n📊 Category distribution:")
print(df_scored['migration_category'].value_counts().sort_index())
print(f"\n📋 Sample categorized rows:")
df_scored[['district_id', 'month', 'migration_score', 'migration_category']].head(12)

✅ Migration categories assigned

📊 Category distribution:
migration_category
High Inflow          39
High Outflow         64
Moderate Inflow     104
Moderate Outflow     33
Name: count, dtype: int64

📋 Sample categorized rows:


,district_id,month,migration_score,migration_category
0,GJ_AMD,2024-01,0.0017,Moderate Inflow
1,GJ_AMD,2024-02,0.0154,High Inflow
2,GJ_AMD,2024-03,0.0105,Moderate Inflow
3,GJ_AMD,2024-04,0.0006,Moderate Inflow
4,GJ_AMD,2024-05,0.0246,High Inflow
5,GJ_AMD,2024-06,0.0202,High Inflow
6,GJ_AMD,2024-07,0.0054,Moderate Inflow
7,GJ_AMD,2024-08,0.0144,Moderate Inflow
8,GJ_AMD,2024-09,0.0211,High Inflow
9,GJ_AMD,2024-10,-0.0023,Moderate Outflow


## Cell 6: Prepare Final Output Schema

In [6]:
"""
Prepare Final Output
--------------------
Select only the required columns in the exact schema order.
Drop all intermediate/helper columns.

Output Schema:
- district_id (string)
- month (string, YYYY-MM)
- migration_score (float)
- migration_category (string)
"""

# Define exact output schema
OUTPUT_COLUMNS = [
    'district_id',
    'month',
    'migration_score',
    'migration_category'
]

# Select only required columns
df_output = df_scored[OUTPUT_COLUMNS].copy()

# Sort by district_id and month for deterministic output
df_output = df_output.sort_values(['district_id', 'month']).reset_index(drop=True)

print(f"✅ Final output prepared: {len(df_output):,} rows")
print(f"\n📊 Columns: {list(df_output.columns)}")
print(f"\n📋 Data types:")
print(df_output.dtypes)
print(f"\n📋 First 12 rows:")
df_output.head(12)

✅ Final output prepared: 240 rows

📊 Columns: ['district_id', 'month', 'migration_score', 'migration_category']

📋 Data types:
district_id            object
month                  object
migration_score       float64
migration_category     object
dtype: object

📋 First 12 rows:


,district_id,month,migration_score,migration_category
0,GJ_AMD,2024-01,0.0017,Moderate Inflow
1,GJ_AMD,2024-02,0.0154,High Inflow
2,GJ_AMD,2024-03,0.0105,Moderate Inflow
3,GJ_AMD,2024-04,0.0006,Moderate Inflow
4,GJ_AMD,2024-05,0.0246,High Inflow
5,GJ_AMD,2024-06,0.0202,High Inflow
6,GJ_AMD,2024-07,0.0054,Moderate Inflow
7,GJ_AMD,2024-08,0.0144,Moderate Inflow
8,GJ_AMD,2024-09,0.0211,High Inflow
9,GJ_AMD,2024-10,-0.0023,Moderate Outflow


## Cell 7: Data Quality Validation

In [7]:
"""
Data Quality Checks
-------------------
Validate all required constraints before export.
"""

checks_passed = 0
total_checks = 6

# Check 1: No NaN values
nan_count = df_output.isna().sum().sum()
if nan_count == 0:
    print("✅ Check 1: No NaN values")
    checks_passed += 1
else:
    print(f"❌ Check 1: Found {nan_count} NaN values")

# Check 2: migration_score is float
if df_output['migration_score'].dtype in ['float64', 'float32']:
    print("✅ Check 2: migration_score is float type")
    checks_passed += 1
else:
    print(f"❌ Check 2: migration_score should be float, got {df_output['migration_score'].dtype}")

# Check 3: migration_category is non-null
if df_output['migration_category'].notna().all():
    print("✅ Check 3: migration_category is non-null")
    checks_passed += 1
else:
    print("❌ Check 3: migration_category has null values")

# Check 4: Valid category values
valid_categories = {'High Inflow', 'Moderate Inflow', 'Moderate Outflow', 'High Outflow'}
actual_categories = set(df_output['migration_category'].unique())
if actual_categories.issubset(valid_categories):
    print(f"✅ Check 4: Valid category values: {actual_categories}")
    checks_passed += 1
else:
    print(f"❌ Check 4: Invalid categories found: {actual_categories - valid_categories}")

# Check 5: Correct column count (exactly 4)
if len(df_output.columns) == 4:
    print("✅ Check 5: Exactly 4 columns (no extra columns)")
    checks_passed += 1
else:
    print(f"❌ Check 5: Expected 4 columns, got {len(df_output.columns)}")

# Check 6: Row count matches input
expected_rows = len(df)
if len(df_output) == expected_rows:
    print(f"✅ Check 6: Row count matches input ({expected_rows} rows)")
    checks_passed += 1
else:
    print(f"❌ Check 6: Row count mismatch. Expected {expected_rows}, got {len(df_output)}")

print(f"\n{'='*50}")
if checks_passed == total_checks:
    print(f"✅ ALL {total_checks} VALIDATION CHECKS PASSED")
else:
    print(f"⚠️ {checks_passed}/{total_checks} checks passed")

✅ Check 1: No NaN values
✅ Check 2: migration_score is float type
✅ Check 3: migration_category is non-null
✅ Check 4: Valid category values: {'Moderate Outflow', 'Moderate Inflow', 'High Outflow', 'High Inflow'}
✅ Check 5: Exactly 4 columns (no extra columns)
✅ Check 6: Row count matches input (240 rows)

✅ ALL 6 VALIDATION CHECKS PASSED


## Cell 8: Export to CSV

In [8]:
"""
Export to CSV
-------------
Save the migration intensity data in CSV format.
Uses snake_case column names.
"""

# Export to CSV (no index column)
df_output.to_csv(OUTPUT_CSV_PATH, index=False)

print(f"✅ CSV exported to: {OUTPUT_CSV_PATH}")
print(f"\n📊 Export Summary:")
print(f"   Total rows: {len(df_output):,}")
print(f"   Districts: {df_output['district_id'].nunique()}")
print(f"   Months: {df_output['month'].nunique()}")
print(f"\n📋 Schema (snake_case):")
for col in df_output.columns:
    sample_val = df_output[col].iloc[0]
    print(f"   {col}: {df_output[col].dtype} (e.g., {sample_val})")

✅ CSV exported to: d:\Projects\ML - DataScience\Saarthi-Net-Data-Pipeline\data\final\migration_intensity.csv

📊 Export Summary:
   Total rows: 240
   Districts: 20
   Months: 12

📋 Schema (snake_case):
   district_id: object (e.g., GJ_AMD)
   month: object (e.g., 2024-01)
   migration_score: float64 (e.g., 0.0017)
   migration_category: object (e.g., Moderate Inflow)


## Cell 9: Export to JSON

In [9]:
"""
Export to JSON
--------------
Save the migration intensity data in JSON format.
Uses camelCase key names as required for frontend consumption.

JSON Format:
[
  {
    "districtId": "RJ_JPR",
    "month": "2024-07",
    "migrationScore": 0.23,
    "migrationCategory": "High Inflow"
  }
]
"""

# Convert to list of dictionaries with camelCase keys
json_records = []
for _, row in df_output.iterrows():
    record = {
        "districtId": row['district_id'],
        "month": row['month'],
        "migrationScore": float(row['migration_score']),
        "migrationCategory": row['migration_category']
    }
    json_records.append(record)

# Write JSON file with proper formatting
with open(OUTPUT_JSON_PATH, 'w', encoding='utf-8') as f:
    json.dump(json_records, f, indent=2, ensure_ascii=False)

print(f"✅ JSON exported to: {OUTPUT_JSON_PATH}")
print(f"\n📊 Export Summary:")
print(f"   Total records: {len(json_records):,}")
print(f"\n📋 Schema (camelCase):")
print(f"   districtId: string")
print(f"   month: string (YYYY-MM)")
print(f"   migrationScore: float")
print(f"   migrationCategory: string")
print(f"\n📄 Sample JSON records:")
print(json.dumps(json_records[:3], indent=2))

✅ JSON exported to: d:\Projects\ML - DataScience\Saarthi-Net-Data-Pipeline\data\final\migration_intensity.json

📊 Export Summary:
   Total records: 240

📋 Schema (camelCase):
   districtId: string
   month: string (YYYY-MM)
   migrationScore: float
   migrationCategory: string

📄 Sample JSON records:
[
  {
    "districtId": "GJ_AMD",
    "month": "2024-01",
    "migrationScore": 0.0017,
    "migrationCategory": "Moderate Inflow"
  },
  {
    "districtId": "GJ_AMD",
    "month": "2024-02",
    "migrationScore": 0.0154,
    "migrationCategory": "High Inflow"
  },
  {
    "districtId": "GJ_AMD",
    "month": "2024-03",
    "migrationScore": 0.0105,
    "migrationCategory": "Moderate Inflow"
  }
]


## Phase 3 Complete

### Output Files
1. `data/final/migration_intensity.csv` (snake_case)
2. `data/final/migration_intensity.json` (camelCase)

### Schema
| Field | CSV Name | JSON Name | Type |
|-------|----------|-----------|------|
| District ID | district_id | districtId | string |
| Month | month | month | string (YYYY-MM) |
| Migration Score | migration_score | migrationScore | float |
| Migration Category | migration_category | migrationCategory | string |

### Category Distribution
- **High Inflow**: Districts with migration_score > 0.015
- **Moderate Inflow**: Districts with score 0.00 to 0.015
- **Moderate Outflow**: Districts with score -0.015 to 0.00
- **High Outflow**: Districts with score < -0.015

### Next Phase
Phase 4 will add peri-urban labels and Phase 5 will add digital exclusion scores.